In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from datetime import datetime
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covid19-global-forecasting-week-4/submission.csv
/kaggle/input/covid19-global-forecasting-week-4/test.csv
/kaggle/input/covid19-global-forecasting-week-4/train.csv


In [2]:
test = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
submission = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/submission.csv")
ForecastId = pd.DataFrame({'ForecastId': submission['ForecastId']})
ForecastId.head(5)

,ForecastId
0,1
1,2
2,3
3,4
4,5


In [3]:
df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
df.head(5)

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [4]:
df['Country_Region'].nunique()

184

In [5]:
region_list = df[['Province_State', 'Country_Region']].drop_duplicates()
region_list

,Province_State,Country_Region
0,NaN,Afghanistan
84,NaN,Albania
168,NaN,Algeria
252,NaN,Andorra
336,NaN,Angola
...,...,...
25872,NaN,Vietnam
25956,NaN,West Bank and Gaza
26040,NaN,Western Sahara
26124,NaN,Zambia


In [6]:
def select_data(train):
    confirm = train[['Date','ConfirmedCases']]
    confirm = confirm.rename(columns={'ConfirmedCases': 'y_data'})
    fatalty = train[['Date','Fatalities']]
    fatalty = fatalty.rename(columns={'Fatalities': 'y_data'})
    confirm.reset_index(drop=True, inplace=True)
    confirm.index = confirm.index + 1
    fatalty.reset_index(drop=True, inplace=True)
    fatalty.index = fatalty.index + 1
    return confirm, fatalty

Exponential growth function:  
<center>$P(t)=P_0e^{kt}$</center>

In [7]:
def exponential_growth_model(t, k, P0):
    t=t/365
    return P0*np.exp(k*t)

Logistic growth model:  
    <center>$P(t)=\frac{KP_0e^{rt}}{K+P_0(e^{rt}-1)}$</center>

In [8]:
def logistic_growth_model(t, K, P0, r):
    # t:time(yr)   t0:initial time    P0:initial_value    K:capacity  r:increase_rate
    t0=0
    t=t/365
    exp_value=np.exp(r*(t-t0))
    return (K*exp_value*P0)/(K+(exp_value-1)*P0)

Linear growth model:  
<center>$P(t)=kt+P_0$</center>


In [9]:
def linear_growth_model(t, k, P0):
    t=t/365
    return k*t+P0

root mean squared logarithmic error  
RMSLE for a single column calculated as:  
<center>$\sqrt{\frac{1}{n}\sum_{i=1}^{n}(log(p_i+1)-log(a_i+1))^2}$</center>

In [10]:
def RMSLE(predict, actual):
    rmsle = np.sqrt(np.mean(np.log((predict+1)/(actual+1))))
    return rmsle

In [11]:
def cal_date_interval(d_start, d_end):
    date_format = "%Y-%m-%d"
    a = datetime.strptime(d_start, date_format)
    b = datetime.strptime(d_end, date_format)
    delta = b - a
    return delta.days

In [12]:
def fit_pred(train):
    t_train = np.linspace(1, len(train['Date']), num=len(train['Date']))
    popt_exp, pcov_exp = curve_fit(exponential_growth_model, t_train, train['y_data'])
    popt_log, pcov_log = curve_fit(logistic_growth_model, t_train, train['y_data'], maxfev=1000)
    
#     Using RMSLE score to decide choose Exponential or Logistic growth model
    t_start = train[train['Date'].isin(['2020-04-02'])].index.tolist()[0]
    t_public_end = t_start + 12
    duration = 13
    t_public_pred = np.linspace(t_start, t_public_end, duration)
    y_actual = train['y_data'][train['Date']>='2020-04-02']
    
    y_exp_pred = exponential_growth_model(t_public_pred, *popt_exp)
    y_log_pred = logistic_growth_model(t_public_pred, *popt_log)
    eval_exp = RMSLE(y_exp_pred, y_actual)
    eval_log = RMSLE(y_log_pred, y_actual)

#     To get final prediction
    t_end = t_start + 42
    t_pred = np.linspace(t_start, t_end, 43)
    if eval_exp < eval_log:
        y_pred = exponential_growth_model(t_pred, *popt_exp)
        eval_public = eval_exp
    else:
        y_pred = logistic_growth_model(t_pred, *popt_log)
        eval_public = eval_log
        
#     plot(train['y_data'], logistic_growth_modellogistic_model(np.linspace(1, 107, 107), *popt))
    return y_pred, eval_public

In [13]:
def unqualify_data(train):
    '''
    When report errors 
    RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 1000.
    Using Linear Regression or Exponential to predict unqualified data
    '''
    
    if (train['y_data'][train.Date == '2020-04-02'].values > 0)[0] == True:
        train = train[train['y_data'] > 0]
    else:
        train = train[train['Date']>='2020-04-02']
    train.reset_index(drop=True, inplace=True)
    train.index = train.index + 1

    t_train = np.linspace(1, len(train['Date']), num=len(train['Date']))
    popt_exp, pcov_exp = curve_fit(exponential_growth_model, t_train, train['y_data'])
    popt_lin, pcov_lin = curve_fit(linear_growth_model, t_train, train['y_data'])
    
    #     Using RMSLE score to decide choose Exponential or Linear growth model
    t_start = train[train['Date'].isin(['2020-04-02'])].index.tolist()[0]
    t_public_end = train.index.values[-1]
    duration = t_public_end - t_start + 1
    t_public_pred = np.linspace(t_start, t_public_end, duration)
    y_actual = train['y_data'][train['Date']>='2020-04-02']
    
    y_exp_pred = exponential_growth_model(t_public_pred, *popt_exp)
    y_lin_pred = linear_growth_model(t_public_pred, *popt_lin)
    eval_exp = RMSLE(y_exp_pred, y_actual)
    eval_lin = RMSLE(y_lin_pred, y_actual)

#     To get final prediction
    interval = cal_date_interval('2020-04-02', '2020-05-14')
    t_pred = np.linspace(t_start, t_start+interval, interval+1)
    if eval_exp < eval_lin:
        y_pred = exponential_growth_model(t_pred, *popt_exp)
        eval_public = eval_exp
    else:
        y_pred = linear_growth_model(t_pred, *popt_lin)
        eval_public = eval_lin
    return y_pred, eval_public   

In [14]:
def plot(actual, pred):
    plt.plot(np.linspace(1, len(actual), len(actual)), actual)
    plt.plot(np.linspace(1, len(pred), len(pred)), pred)
    plt.show()

In [15]:
c_pred_global = pd.DataFrame(columns=None)
f_pred_global = pd.DataFrame(columns=None)
c_eval = pd.DataFrame(columns=None)
f_eval = pd.DataFrame(columns=None)

for i in range(len(region_list)):
#     train = df[np.logical_and(np.logical_or(pd.isnull(df['Province_State']), df['Province_State']==region_list.iloc[i][0]), 
#                           df['Country_Region']==region_list.iloc[i][1])]
    train = df.iloc[84*i:84*(i+1), :]
    confirm, fatalty = select_data(train)
    
    try: 
        confirm_pred, confirm_eval = fit_pred(confirm)
    except:
        confirm_pred, confirm_eval = unqualify_data(confirm)
#     c_eval = c_eval.append(pd.DataFrame([confirm_eval],columns={'RMSLE'}, index=i))
    c_eval = c_eval.append(pd.DataFrame([confirm_eval],columns={'RMSLE'}))
    c_pred_global = c_pred_global.append(pd.DataFrame(confirm_pred), ignore_index=True)

    try: 
        fatalty_pred, fatalty_eval = fit_pred(fatalty)
    except:
        fatalty_pred, fatalty_eval = unqualify_data(fatalty)
#     f_eval = f_eval.append(pd.DataFrame([fatalty_eval],columns={'RMSLE'}, index=i))
    f_eval = f_eval.append(pd.DataFrame([fatalty_eval],columns={'RMSLE'}))
    f_pred_global = f_pred_global.append(pd.DataFrame(fatalty_pred), ignore_index=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in sqrt
  
/opt/conda/lib/python3.6/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [16]:
# Evalutaion of public data
c_eval.sort_values(by=['RMSLE'])

,RMSLE
0,0.000015
0,0.013770
0,0.015474
0,0.018200
0,0.018603
...,...
0,NaN
0,NaN
0,NaN
0,NaN


In [17]:
f_eval.sort_values(by=['RMSLE'])

,RMSLE
0,0.0
0,0.0
0,0.0
0,0.0
0,0.0
...,...
0,NaN
0,NaN
0,NaN
0,NaN


In [18]:
c_pred_global = c_pred_global.rename(columns={0:'ConfirmedCases'})
c_pred_global

,ConfirmedCases
0,238.215366
1,269.050395
2,302.310550
3,337.804325
4,375.251369
...,...
13454,18.661080
13455,18.671662
13456,18.680759
13457,18.688577


In [19]:
f_pred_global = f_pred_global.rename(columns={0:'Fatalities'})
f_pred_global

,Fatalities
0,5.998856
1,6.916607
2,7.939261
3,9.068137
4,10.301396
...,...
13454,41.212701
13455,45.256945
13456,49.698054
13457,54.574974


In [20]:
# when axis=1, concat horizontaly
output = pd.concat([c_pred_global, f_pred_global], axis=1) 
output = pd.concat([ForecastId, output], axis=1) 
output

,ForecastId,ConfirmedCases,Fatalities
0,1,238.215366,5.998856
1,2,269.050395,6.916607
2,3,302.310550,7.939261
3,4,337.804325,9.068137
4,5,375.251369,10.301396
...,...,...,...
13454,13455,18.661080,41.212701
13455,13456,18.671662,45.256945
13456,13457,18.680759,49.698054
13457,13458,18.688577,54.574974


In [21]:
output.to_csv('submission.csv', header=True, index=False) 